In [38]:
!pip install transformers datasets evaluate nlpaug


In [39]:
from datasets import load_dataset
import pandas as pd
import nlpaug.augmenter.word as naw
from collections import Counter


In [40]:

ds = load_dataset("dair-ai/emotion", "split")

In [41]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [42]:
# Convert train split to DataFrame
train_df = pd.DataFrame(ds["train"])

# Display first few rows
print(train_df.head())

# Class distribution
counter = Counter(train_df["label"])
label_names = ds["train"].features["label"].names
for label_id, count in counter.items():
    print(f"{label_names[label_id]:<8}: {count}")


                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3
sadness : 4666
anger   : 2159
love    : 1304
surprise: 572
fear    : 1937
joy     : 5362


In [43]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased",do_lower_case=True)

In [44]:
max_length = 128

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )


In [45]:
tokenized_datasets = ds.map(
    tokenize_batch,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [46]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest"
)


In [47]:
tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)
train_ds = tokenized_datasets["train"]
val_ds   = tokenized_datasets["validation"]
test_ds  = tokenized_datasets["test"]


In [48]:
from transformers import (
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
import evaluate


In [49]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,                # our 6 emotion classes
    ignore_mismatched_sizes=True # safe if vocab sizes differ
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
accuracy_metric = evaluate.load("accuracy")
f1_metric       = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    f1  = f1_metric.compute(predictions=preds, references=labels, average="macro")
    return {"accuracy": acc["accuracy"], "f1_macro": f1["f1"]}


In [51]:
training_args = TrainingArguments(
    output_dir="emotion-distilbert",
    do_train=True,
    do_eval=True,
    eval_steps=500,
    save_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=50,
    fp16=True,
    gradient_checkpointing=False
)


In [52]:
!pip install peft


In [53]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,

    # Tell PEFT which linear modules to wrap with LoRA
    target_modules=["q_lin", "v_lin"]  # for DistilBERT’s attention projections
)
model = get_peft_model(model, lora_config)


In [54]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total params: {total_params:,}")
print(f"Trainable params: {trainable_params:,}")  # should be just the LoRA adapters


Total params: 67,700,748
Trainable params: 742,662


In [55]:
from transformers import Trainer

trainer = Trainer(
    model=model,                # your LoRA-wrapped model
    args=training_args,         # the args that worked for you
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-55-77475f9956c7>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
train_result = trainer.train()


Step,Training Loss
50,1.805000
100,1.752200
150,1.662000
200,1.564500
250,1.492900
300,1.329300
350,1.243000


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd

# 1. Get predictions on the test split
preds_output = trainer.predict(test_ds)
pred_labels = np.argmax(preds_output.predictions, axis=-1)
true_labels = preds_output.label_ids

# 2. Map numeric IDs to emotion names
label_names = ds["train"].features["label"].names

# 3. Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:\n", cm_df)

# 4. Classification Report
cr = classification_report(true_labels, pred_labels, target_names=label_names)
print("\nClassification Report:\n", cr)


In [ ]:
model.save_pretrained("Deon_emotion-model")
tokenizer.save_pretrained("Deon-emotion-model")


In [ ]:
from huggingface_hub import login
login()



In [ ]:
model.push_to_hub("chinesemusk/Deon_emotion-model")
tokenizer.push_to_hub("chinesemusk/Deon-emotion-model")

In [ ]:
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
    pipeline
)
from peft import PeftModel

# 1. Load the base model with the correct label count
base = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6
)

# 2. Attach your LoRA adapters from your Hub repo
model = PeftModel.from_pretrained(
    base,
    "chinesemusk/Deon_emotion-model"  # your model repo name
)

# 3. Load the matching tokenizer (from the same repo)
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "chinesemusk/Deon-emotion-model"
)

# 4. Build the pipeline on that exact model+tokenizer
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

# 5. Test it!
print(classifier("I just got promoted at work — I'm over the moon!"))


In [ ]:
import torch
import torch.nn.functional as F

# 1. Put model in eval mode & move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

# 2. Define label names
label_names = ds["train"].features["label"].names

def predict_emotion(text: str):
    # Tokenize & convert to tensors
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    # Forward pass (LoRA adapters are active)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Softmax → probabilities
    probs = F.softmax(logits, dim=-1).squeeze().cpu().tolist()

    # Pair labels & scores
    return dict(zip(label_names, probs))

# 3. Try it out
print(predict_emotion("I can't believe how beautiful the sunset is!"))


In [ ]:
merged_model = model.merge_and_unload()
merged_model.push_to_hub("chinesemusk/Deon_emotion-model-full")
tokenizer.push_to_hub("chinesemusk/Deon-emotion-model-full")


In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="chinesemusk/Deon_emotion-model-full",
    tokenizer="chinesemusk/Deon-emotion-model-full",
    top_k=None  # returns all class scores
)

text = "I'm feeling really hopeful and excited today!"
results = classifier(text)
print(results)


In [ ]:
!pip install gradio
import gradio as gr
from transformers import pipeline

# Load your model and tokenizer from Hugging Face
classifier = pipeline(
    "text-classification",
    model="chinesemusk/Deon_emotion-model-full",
    tokenizer="chinesemusk/Deon-emotion-model-full",
    top_k=None  # to show scores for all classes
)

# Define the interface function
def classify_emotion(text):
    results = classifier(text)[0]
    # Format results for display
    return {res['label']: float(f"{res['score']:.3f}") for res in results}

# Launch the Gradio UI
gr.Interface(
    fn=classify_emotion,
    inputs=gr.Textbox(lines=2, placeholder="Enter a sentence..."),
    outputs=gr.Label(num_top_classes=6),
    title="Deon Emotion Classifier",
    description="Enter a sentence to detect the expressed emotion (joy, love, sadness, anger, fear, surprise)."
).launch()


In [ ]:
# app.py
import gradio as gr
from transformers import pipeline

# Load pipeline from your fine-tuned model
classifier = pipeline(
    "text-classification",
    model="chinesemusk/Deon_emotion-model-full",
    tokenizer="chinesemusk/Deon-emotion-model-full",
    top_k=None,  # Show all class probabilities
    device=0  # Use GPU if available
)

# Define prediction function
def predict_emotion(text):
    results = classifier(text)[0]
    # Format as label: probability%
    return {res["label"]: float(f"{res['score']:.4f}") for res in results}

# Build Gradio interface
demo = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Textbox(lines=3, placeholder="Enter text here..."),
    outputs=gr.Label(num_top_classes=6),
    title="Deon Emotion Classifier",
    description="Predicts emotion from English text using a fine-tuned DistilBERT model. Try inputs like 'I love this moment' or 'I'm scared.'",
    examples=[
        ["I love this!"],
        ["I feel so empty today..."],
        ["This is amazing!"],
        ["I'm really angry right now"],
        ["What a shocking turn of events!"]
    ]
)

if __name__ == "__main__":
    demo.launch()
